* 실행환경: Colab
* python version: 3.9.9
* gymnasium: 1.0.0
* torch: 2.5.1+cu118

In [ ]:
# %%capture
# %%bash
# pip install gymnasium

In [ ]:
import gymnasium as gym
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from gymnasium.envs.registration import register
from gymnasium.envs.classic_control.acrobot import AcrobotEnv
import warnings

warnings.filterwarnings('ignore')

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion() # matplotlib를 interactive mode로 설정 -> 그래프를 실시간으로 업데이트할 수 있도록 함

device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print(device)

cuda


In [3]:
class CustomAcrobotEnv(AcrobotEnv):
    def __init__(self, link1_length=1.0, link2_length=1.0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.LINK_LENGTH_1 = link1_length
        self.LINK_LENGTH_2 = link2_length

register(
    id='CustomAcrobot-v0',
    entry_point=__name__ + ':CustomAcrobotEnv',
    kwargs={'link1_length': 1.0, 'link2_length': 1.0},
    max_episode_steps=500
)

# Load the custom environment
env = gym.make('CustomAcrobot-v0')
print(env.spec)

EnvSpec(id='CustomAcrobot-v0', entry_point='__main__:CustomAcrobotEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=500, order_enforce=True, disable_env_checker=False, kwargs={'link1_length': 1.0, 'link2_length': 1.0}, namespace=None, name='CustomAcrobot', version=0, additional_wrappers=(), vector_entry_point=None)


In [4]:
SEED = 42

env.action_space.seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [5]:
def plot_rewards(show_result=False):

    rewards_t = torch.tensor(episode_rewards, dtype=torch.float)

    plt.clf()
    plt.title('Result')
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.plot(rewards_t.numpy(), label='Episode Reward')

    # 100 에피소드 이동 평균 계산 및 플롯
    if len(rewards_t) >= 100:
        means = rewards_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy(), label='100 Episode Avg', linestyle='--')

    plt.legend()
    plt.pause(0.001)

    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [6]:
class DuelingDQN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DuelingDQN, self).__init__()
        self.fc1 = nn.Linear(n_observations, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 128)

        # DuelingDQN 적용
        # 상태값 계산을 위한 레이어
        self.fc_value = nn.Sequential(
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(64, 1)
            )

        # 행동 이점 계산을 위한 레이어
        self.fc_advantage = nn.Sequential(
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(64, n_actions)  # 행동 이점은 행동 공간 크기
        )

    def forward(self, x):
        x = torch.nn.functional.leaky_relu(self.fc1(x))
        x = torch.nn.functional.leaky_relu(self.fc2(x))
        x = torch.nn.functional.leaky_relu(self.fc3(x))

        # 상태값과 행동 이점 계산
        value = self.fc_value(x)
        advantage = self.fc_advantage(x)

        # 최종 Q값 계산
        q_values = value + advantage - advantage.mean(1, keepdim=True)
        return q_values

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class PrioritizedReplayMemory:
    def __init__(self, capacity, alpha=0.6):
        self.capacity = capacity
        self.memory = deque([], maxlen=capacity)
        self.priorities = deque([], maxlen=capacity)
        self.alpha = alpha

    def push(self, *args):
        # 새로운 경험 추가
        max_priority = max(self.priorities, default=1.0) # 가장 높은 우선순위로 초기화
        self.memory.append(Transition(*args))
        self.priorities.append(max(1e-5, float(max_priority)))

    def sample(self, batch_size, beta=0.4):
        priorities = np.array(self.priorities, dtype=np.float32).flatten()  # 1차원 배열로 변환

        if priorities.sum() == 0:
            raise ValueError("All priorities are zero.")  # 우선순위 합이 0인 경우 예외 처리

        # 우선순위를 확률로 변환
        probabilities = priorities ** self.alpha # alpha는 우선순위의 영향력을 제어
        probabilities /= probabilities.sum()

        # 우선순위(확률)에 따라 샘플 선택
        indices = np.random.choice(len(self.memory), batch_size, p=probabilities)
        samples = [self.memory[idx] for idx in indices]

        # 중요도 가중치 계산
        total = len(self.memory)
        weights = (total * probabilities[indices]) ** (-beta)
        weights /= weights.max()  # 가중치 정규화
        weights = torch.tensor(weights, dtype=torch.float32, device=device)

        return samples, weights, indices


    def update_priorities(self, indices, priorities):
            # 우선순위 업데이트
            for idx, priority in zip(indices, priorities):
                self.priorities[idx] = max(1e-5, float(priority)) # 최소값 보장

    def __len__(self):
        return len(self.memory)

def optimize_model():
    if len(memory) < BATCH_SIZE:
        return

    samples, weights, indices = memory.sample(BATCH_SIZE, beta)
    batch = Transition(*zip(*samples))

    # 다음 상태 중 None이 아닌 것만 선택
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None]).to(device)
    # 모든 상태를 (배치 크기, 채널 수, 높이, 너비) 형식으로 연결
    state_batch = torch.cat(batch.state).to(device)
    action_batch = torch.cat(batch.action).to(device) # 각 상태에 대해 선택된 행동의 인덱스
    reward_batch = torch.cat(batch.reward).to(device)

    # Q값 계산
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # 다음 상태의 Q값 계산
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        # Double DQN으로 수정
        next_actions = policy_net(non_final_next_states).max(1).indices.unsqueeze(1)
        next_state_values[non_final_mask] = target_net(non_final_next_states).gather(1, next_actions).squeeze(1)

    # 기대 Q값 계산
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    td_errors = (state_action_values - expected_state_action_values.unsqueeze(1)).abs().detach()
    td_errors = td_errors.cpu().numpy()
    td_errors = np.maximum(td_errors, 1e-5)  # 최소값 보장
    memory.update_priorities(indices, td_errors)

    # Huber 손실 함수와 중요도 가중치 적용
    criterion = nn.SmoothL1Loss() # 작은 오차에는 MSE로 동작, 큰 오차에는 L1 Loss로 동작
    
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    loss = (weights * loss).mean() # 중요도 가중치 적용

    # 모델 최적화
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1

    state = state.to(device)

    # 초기 eps_threshold는 1.0. steps_done이 증가함에 따라 점진적으로 감소하여 결국 Q-value가 가장 큰 행동을 선택.
    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(1).indices.view(1,1)

    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)


In [7]:
# 학습 파라미터
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 1.0
EPS_END = 0.1
EPS_DECAY = 20000 # 크게 할수록 캄험이 더 오래 지속 -> 새로운 더 많이 탐색
TAU = 0.005
LR = 5e-4
memory_capacity = 50000 # 크게 할수록 에이전트가 더 많은 경험을 저장

n_actions = env.action_space.n # 3

state, info = env.reset()
n_observations = len(state) # 6

policy_net = DuelingDQN(n_observations, n_actions).to(device)
target_net = DuelingDQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = PrioritizedReplayMemory(memory_capacity)

steps_done = 0

alpha = 0.6
beta = 0.4

episode_rewards = []
score = 0.0

if torch.cuda.is_available() or torch.backends.mps.is_available():
    num_episodes = 1000
else:
    num_episodes = 50

In [ ]:
for episode in range(num_episodes+1):

    state, info = env.reset(seed=SEED)
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

    episode_reward = 0

    for t in count(): # 무한 반복
        if episode < 10:  # 초기 10 에피소드는 무조건 랜덤 행동
            action = torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)
        else:
            action = select_action(state)
            
        next_state, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        score += reward.item()
        episode_reward += reward.item()

        if not done:
            next_state = torch.tensor(next_state, dtype=torch.float32, device=device).unsqueeze(0)
        else:
            next_state = None

        memory.push(state, action, next_state, reward)
        state = next_state

        # policy network 업데이트
        optimize_model()

        # target network 업데이트
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_rewards.append(episode_reward)
            break

    if episode % 10 == 0 and episode != 0:
        plot_rewards()

        
print('Complete')
plot_rewards(show_result=True)
plt.ioff()
plt.show()
print('Average Reward: {}'.format(sum(episode_rewards)/num_episodes))

In [ ]:
episode_rewards